<a href="https://colab.research.google.com/github/njugunah800/Building-With-Gemini/blob/main/Copy_of_building_with_gemini_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Important Packages

In [1]:
%pip install -U -q 'google-genai' #installing
%pip install -q chromadb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.6 MB/s eta 0:00:0

In [3]:

#@title Import Python Libraries
# General data science libraries
import pandas as pd
import numpy as np

# Pulling data from APIs, parsing JSON
import requests
import json

# Interfacing w/ Cloud Storage from Python
from google.cloud import storage

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import HTML, Image, Markdown
from google import genai
from google.genai import types


# Create An API Key
- Make sure you are logged in to your google account for this next step

* Create API key in AI studio. https://aistudio.google.com/apikey  

In [8]:
#get API key from env

from google.colab import userdata

GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# CHOOSE YOUR MODEL



# We'll Use the Gemini Flash Models

In [9]:
MODEL_ID = "gemini-2.0-flash" # @param ["gemini-1.5-flash-latest","gemini-2.0-flash-lite","gemini-2.0-flash","gemini-2.5-pro-exp-03-25"]


In [10]:
client = genai.Client(api_key=GOOGLE_API_KEY) #instance of gen ai client

how we upload the video

In [11]:
#fn--Upload the video
import time

def upload_video(video_file_name):
  video_file = client.files.upload(file=video_file_name)

  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name)

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + video_file.uri)

  return video_file

In [ ]:
#Video Analysis with Gemini

In [12]:
vid_file ="https://sporty-clips.mlb.com/MTYyVlZfV0ZRVkV3dEdEUT09X0F3ZFJCVlpTWDFBQUFWY0tYd0FBVkFJRUFGZ0FCVlFBQUFCVEJ3b0RBZ0JSVVF0Uw==.mp4" #@param {type:"string"}
!wget {vid_file} -O home_run.mp4 -q # Directly download and save as home_run.mp4
home_run_video = upload_video('home_run.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/j5xtq37b8zcx


In [13]:
HTML(f"""<video width="640" height="360" controls>
          <source src="{vid_file}" type="video/mp4">
          Your browser does not support the video tag.
        </video>""")

In [14]:
system_instruction_prompt = "You are a baseball analytics systems, you analyze the video frame by frame and you give the best analysis possible, clear and concise but verbose on request." # @param {type:"string"}
prompt = "What's happening in the video, give me stats from the statcast." # @param ["Give me a table of my items and notes", "Help me come up with a selling pitch for my potteries"] {"allow-input":true}

video = home_run_video # @param

response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    config = types.GenerateContentConfig(
        system_instruction= system_instruction_prompt,

    )
)

In [16]:
Markdown(response.text)

Certainly. Here's the analysis of the play from the clip:

**Event:** Freddie Freeman hits a walk-off home run, winning Game 1 of the World Series for the Dodgers.

**Pitch:**

*   Pitch type: Four-seam fastball
*   Velocity: 93 mph

The ball was hit to right field and went out of the park. The score was tied 3-3 with the Dodgers on the verge of losing.

# Use JSON Mode

In [ ]:
import typing_extensions as typing

In [ ]:
class ResponseSchema(typing.TypedDict):
    play_description: str
    ball_type: str
    pitch_speed: str
    exit_velocity: str
    launch_angle: str
    launch_direction: str

In [ ]:
response = client.models.generate_content(
    model=MODEL_ID,
    contents=[
        video,
        prompt,
    ],
    config = types.GenerateContentConfig(
        system_instruction= system_instruction_prompt,
        response_mime_type="application/json",
        response_schema = ResponseSchema),
    )


In [ ]:
print(json.dumps(json.loads(response.text), indent=4))

{
    "play_description": "Freddie Freeman hits a walk-off home run to right field",
    "ball_type": "Four-Seam Fastball",
    "pitch_speed": "93 MPH",
    "exit_velocity": "99 MPH",
    "launch_angle": "28 Degrees",
    "launch_direction": "Right Field"
}


# Use Embeddings For Vector Search

In [ ]:
#@title Configure a GenAI instance with your API KEY
import google.generativeai as genai


genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp


In [ ]:

DOCUMENT1 = "Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: The car will blow cool air into the car. Heat: The car will blow warm air into the car. Defrost: The car will blow warm air onto the windshield to defrost it."
DOCUMENT2 = "Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the \"Navigation\" icon to get directions to your destination or touch the \"Music\" icon to play your favorite songs."
DOCUMENT3 = "Shifting Gears Your Googlecar has an automatic transmission. To shift gears, simply move the shift lever to the desired position.  Park: This position is used when you are parked. The wheels are locked and the car cannot move. Reverse: This position is used to back up. Neutral: This position is used when you are stopped at a light or in traffic. The car is not in gear and will not move unless you press the gas pedal. Drive: This position is used to drive forward. Low: This position is used for driving in snow or other slippery conditions."

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [ ]:
from chromadb import Documents, EmbeddingFunction, Embeddings, chromadb


In [ ]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]
  #to be implemented to avoid deprecation
  '''
  def __init__(self, input: Documents) -> Embeddings:
    model = 'models/embedding-001'
    title = "Custom query"
    return genai.embed_content(model=model,
                                content=input,
                                task_type="retrieval_document",
                                title=title)["embedding"]
                                '''


How it works it transforms string using a proprietary vector function to an array of vectors, and stores the vectors and original content.

In [ ]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())
  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

In [ ]:
db = create_chroma_db(documents, "googlecarsdatabase_1")

<ipython-input-63-104ad011433a>:3: DeprecationWarning: The class GeminiEmbeddingFunction does not implement __init__. This will be required in a future version.
  db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())


In [ ]:
db.get(include=["embeddings"])

{'ids': ['0', '1', '2'],
 'embeddings': array([[-0.02099494, -0.03876613, -0.05423354, ...,  0.00865673,
         -0.05088616,  0.02915585],
        [ 0.0174108 , -0.04757163, -0.02510802, ...,  0.03530323,
         -0.03800508,  0.02270922],
        [-0.03194406, -0.02328151, -0.03803354, ..., -0.02371154,
         -0.00614928,  0.03037698]]),
 'documents': None,
 'uris': None,
 'included': ['embeddings'],
 'data': None,
 'metadatas': None}

In [ ]:
db.get(include=["documents"])

{'ids': ['0', '1', '2'],
 'embeddings': None,
 'documents': ['Operating the Climate Control System  Your Googlecar has a climate control system that allows you to adjust the temperature and airflow in the car. To operate the climate control system, use the buttons and knobs located on the center console.  Temperature: The temperature knob controls the temperature inside the car. Turn the knob clockwise to increase the temperature or counterclockwise to decrease the temperature. Airflow: The airflow knob controls the amount of airflow inside the car. Turn the knob clockwise to increase the airflow or counterclockwise to decrease the airflow. Fan speed: The fan speed knob controls the speed of the fan. Turn the knob clockwise to increase the fan speed or counterclockwise to decrease the fan speed. Mode: The mode button allows you to select the desired mode. The available modes are: Auto: The car will automatically adjust the temperature and airflow to maintain a comfortable level. Cool: 

In [ ]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage


Now the search query is transformed in to a set of vectors and compared and this function gets the closeset, document using function :
> `[R = A - S]` reursively `where R the result is the closest matched doc` , `S represents the search string` and  `A is a doc from the database`



In [ ]:
 #@title This a utility function
 #@markdown it helps the application create a prompt to query the AI by joining the QUESTION to the PASSSAGE

def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'
  ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
 #@title Get tht Passage

passage = get_relevant_passage("touch screen features", db)
Markdown(passage)

Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the "Navigation" icon to get directions to your destination or touch the "Music" icon to play your favorite songs.

In [ ]:

query = "How do you use the touchscreen in the Google car?" #@param
prompt = make_prompt(query, passage)
Markdown('''
# This is the prompt(Passage + Query)
'''+prompt)


# This is the prompt(Passage + Query)
You are a helpful and informative bot that answers questions using text from the reference passage included below.  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information.  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and  strike a friendly and converstional tone.  If the passage is irrelevant to the answer, you may ignore it.
 QUESTION: 'How do you use the touchscreen in the Google car?'
 PASSAGE: 'Your Googlecar has a large touchscreen display that provides access to a variety of features, including navigation, entertainment, and climate control. To use the touchscreen display, simply touch the desired icon.  For example, you can touch the Navigation icon to get directions to your destination or touch the Music icon to play your favorite songs.'
 ANSWER:
 

In [ ]:
model = genai.GenerativeModel(MODEL_ID)
answer = model.generate_content(prompt)
Markdown('''
# Gemini Says:
'''+answer.text)


# Gemini Says:
The Googlecar has a large touchscreen display that gives you access to many different features, including navigation, entertainment, and climate control, and you can use it by simply touching the icon for the feature you want to use.
